In [1]:
import pandas as pd
data = pd.read_csv("./savenocut/data_clean.csv")

In [2]:
data = data.drop(data[data["label"] == 2].index).reset_index(drop = True)

In [3]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2, random_state=10)

In [4]:
#段詞
from inlp.convert import chinese
from ckip import CkipSegmenter
import json

train_recovery = []
num = 0

for i in train["recovery"]:
    i = chinese.s2t(i) #簡體轉繁體
    recovery = CkipSegmenter().seg(i).tok
    train_recovery.append(recovery)
    print(num, "done")
    num = num + 1
    
# data = data.drop(data.index[error]).reset_index(drop = True)
# data.to_csv('./savenocut/data_clean.csv', index=False, encoding='utf-8-sig')

    #保存list
c_list = json.dumps(train_recovery)
a = open(r"./savenocut/list_save.txt", "w",encoding='UTF-8')
a.write(c_list)
a.close()

0 done
1 done
2 done
3 done
4 done
5 done
6 done
7 done
8 done
9 done
10 done
11 done
12 done
13 done
14 done
15 done
16 done
17 done
18 done
19 done
20 done
21 done
22 done
23 done
24 done
25 done
26 done
27 done
28 done
29 done
30 done
31 done
32 done
33 done
34 done
35 done
36 done
37 done
38 done
39 done
40 done
41 done
42 done
43 done
44 done
45 done
46 done
47 done
48 done
49 done
50 done
51 done
52 done
53 done
54 done
55 done
56 done
57 done
58 done
59 done
60 done
61 done
62 done
63 done
64 done
65 done
66 done
67 done
68 done
69 done
70 done
71 done
72 done
73 done
74 done
75 done
76 done
77 done
78 done
79 done
80 done
81 done
82 done
83 done
84 done
85 done
86 done
87 done
88 done
89 done
90 done
91 done
92 done
93 done
94 done
95 done
96 done
97 done
98 done
99 done
100 done
101 done
102 done
103 done
104 done
105 done
106 done
107 done
108 done
109 done
110 done
111 done
112 done
113 done
114 done
115 done
116 done
117 done
118 done
119 done
120 done
121 done
122 done
123

In [5]:
#產生一個token_list的Word2Vec
from gensim.models import Word2Vec

model = Word2Vec(train_recovery, window=5, size=500, sg=1, iter=50, min_count=1)
#儲存他
model.save("./savenocut/model_Word2Vec")

In [6]:
from tensorflow.python.keras.layers import Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np

embedding_matrix = np.zeros((len(model.wv.vocab.items()) + 1, model.vector_size))
word2idx = {}
PADDING_LENGTH = 500
vocab_list = [(word, model.wv[word]) for word, _ in model.wv.vocab.items()]

for i, vocab in enumerate(vocab_list):
    word, vec = vocab
    embedding_matrix[i + 1] = vec
    word2idx[word] = i + 1

embedding_layer = Embedding(input_dim=embedding_matrix.shape[0],
                            output_dim=embedding_matrix.shape[1],
                            weights=[embedding_matrix],
                            trainable=False)

#### mapping to index
def text_to_index(corpus):
    new_corpus = []
    for doc in corpus:
        new_doc = []
        for word in doc:
            try:
                new_doc.append(word2idx[word])
            except:
                new_doc.append(0)
        new_corpus.append(new_doc)
    return np.array(new_corpus)

def new_model():
    model = Sequential()
    model.add(embedding_layer)
    model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(2, activation='softmax'))
    
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [7]:
#### 加上label
X = text_to_index(train_recovery)
X = pad_sequences(X, maxlen=PADDING_LENGTH)

Y = to_categorical(train["label"])

<ipython-input-6-0467c45ce11a>:32: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(new_corpus)


In [8]:
#開始跑
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Activation, LSTM

lstm = new_model()
lstm.fit(x=X, y=Y, batch_size=100, epochs=100)

Epoch 1/100
7/7 [==============================] - 76s 11s/step - loss: 0.7042 - accuracy: 0.4762
Epoch 2/100
7/7 [==============================] - 93s 13s/step - loss: 0.6461 - accuracy: 0.6598
Epoch 3/100
7/7 [==============================] - 100s 14s/step - loss: 0.6167 - accuracy: 0.6604
Epoch 4/100
7/7 [==============================] - 107s 15s/step - loss: 0.5953 - accuracy: 0.6873
Epoch 5/100
7/7 [==============================] - 122s 18s/step - loss: 0.5718 - accuracy: 0.6983
Epoch 6/100
7/7 [==============================] - 132s 19s/step - loss: 0.5213 - accuracy: 0.7249
Epoch 7/100
7/7 [==============================] - 145s 21s/step - loss: 0.5177 - accuracy: 0.7219
Epoch 8/100
7/7 [==============================] - 148s 21s/step - loss: 0.5087 - accuracy: 0.7597
Epoch 9/100
7/7 [==============================] - 152s 21s/step - loss: 0.4740 - accuracy: 0.7614
Epoch 10/100
7/7 [==============================] - 153s 22s/step - loss: 0.4327 - accuracy: 0.8186
Epoch 11/10

7/7 [==============================] - 215s 30s/step - loss: 0.0042 - accuracy: 1.0000
Epoch 84/100
7/7 [==============================] - 206s 30s/step - loss: 0.0070 - accuracy: 0.9978
Epoch 85/100
7/7 [==============================] - 210s 30s/step - loss: 0.0043 - accuracy: 0.9989
Epoch 86/100
7/7 [==============================] - 210s 30s/step - loss: 0.0041 - accuracy: 0.9996
Epoch 87/100
7/7 [==============================] - 214s 31s/step - loss: 0.0072 - accuracy: 0.9973
Epoch 88/100
7/7 [==============================] - 215s 31s/step - loss: 0.0070 - accuracy: 0.9966
Epoch 89/100
7/7 [==============================] - 217s 31s/step - loss: 0.0036 - accuracy: 0.9978
Epoch 90/100
7/7 [==============================] - 218s 31s/step - loss: 0.0049 - accuracy: 1.0000
Epoch 91/100
7/7 [==============================] - 215s 31s/step - loss: 0.0032 - accuracy: 1.0000
Epoch 92/100
7/7 [==============================] - 219s 31s/step - loss: 0.0045 - accuracy: 0.9966
Epoch 93/100


In [9]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Activation, LSTM

#儲存他
# save model
lstm.save_weights('./savenocut/lstm.h5')
model_json = lstm.to_json()
with open('./savenocut/lstm.json', "w") as json_file:
    json_file.write(model_json)

### 未來在使用他

In [ ]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Activation, LSTM
import tensorflow as tf
# load model
json_file = open('./savenocut/lstm.json', 'r')
lstm_loaded_model_json = json_file.read()
lstm = tf.keras.models.model_from_json(lstm_loaded_model_json)
lstm.load_weights('./savenocut/lstm.h5')

In [10]:
test_text = test["recovery"].tolist()

In [11]:
X_test = text_to_index(test_text)
X_test = pad_sequences(X_test, maxlen=PADDING_LENGTH)
Y_preds = lstm.predict(X_test)
print(Y_preds)
Y_preds_label = np.argmax(Y_preds, axis=1)
print(Y_preds_label)

<ipython-input-6-0467c45ce11a>:32: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(new_corpus)


[[1.46613549e-02 9.85338688e-01]
 [3.32120389e-01 6.67879641e-01]
 [6.28577603e-04 9.99371469e-01]
 [7.42229223e-02 9.25777078e-01]
 [5.14579892e-01 4.85420167e-01]
 [7.84674048e-01 2.15325922e-01]
 [2.03694761e-01 7.96305299e-01]
 [8.55899677e-02 9.14409995e-01]
 [7.70310283e-01 2.29689747e-01]
 [5.33438213e-02 9.46656168e-01]
 [9.99988317e-01 1.17127238e-05]
 [6.69539198e-02 9.33046103e-01]
 [9.94918525e-01 5.08141285e-03]
 [2.51891047e-01 7.48108983e-01]
 [4.94967587e-02 9.50503170e-01]
 [1.59611091e-01 8.40388894e-01]
 [1.91904232e-01 8.08095813e-01]
 [9.99970794e-01 2.91808137e-05]
 [9.99916196e-01 8.37758926e-05]
 [9.74314570e-01 2.56853905e-02]
 [4.98276234e-01 5.01723766e-01]
 [6.26564384e-01 3.73435676e-01]
 [9.04566705e-01 9.54332799e-02]
 [9.97248709e-01 2.75132083e-03]
 [5.69686651e-01 4.30313349e-01]
 [7.82165094e-04 9.99217868e-01]
 [9.99890804e-01 1.09127468e-04]
 [9.97657418e-01 2.34259525e-03]
 [2.64433939e-02 9.73556638e-01]
 [2.56239716e-02 9.74376023e-01]
 [2.726734

In [4]:
y_test = test["label"]

In [6]:
type(y_test)

pandas.core.series.Series

In [13]:
#看評價
from sklearn.metrics import classification_report
print(classification_report(y_test, Y_preds_label))

              precision    recall  f1-score   support

           0       0.67      0.67      0.67        99
           1       0.57      0.57      0.57        76

    accuracy                           0.62       175
   macro avg       0.62      0.62      0.62       175
weighted avg       0.62      0.62      0.62       175



## 匯入所有的資料

In [15]:
import pandas as pd
all_data = pd.read_csv("C:/Users/DA01001/Desktop/tebproject/0205NLP/no_cut_recovery_all.csv")

In [16]:
all_text = all_data["recovery"].tolist()

In [17]:
len(all_text)

25470

### 預測他

In [21]:
X_test = text_to_index(all_text)
X_test = pad_sequences(X_test, maxlen=PADDING_LENGTH)
Y_preds = lstm.predict(X_test)
print(Y_preds)
Y_preds_label = np.argmax(Y_preds, axis=1)
print(Y_preds_label)

<ipython-input-6-0467c45ce11a>:32: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(new_corpus)


[[9.7349685e-01 2.6503149e-02]
 [9.9952352e-01 4.7644525e-04]
 [2.6068450e-03 9.9739313e-01]
 ...
 [9.9753040e-01 2.4696779e-03]
 [2.3602636e-01 7.6397365e-01]
 [4.0716209e-04 9.9959284e-01]]
[0 0 1 ... 0 1 1]


In [22]:
len(Y_preds_label)

25470

In [ ]:
Y_preds_label

### 把剛剛預測的資料加進原始資料

In [23]:
all_data["Y_preds_label"] = Y_preds_label

In [24]:
all_data.to_csv('./savenocut/preds_label.csv', index=False, encoding='utf-8-sig')

### group articleID 計算正向有幾個

In [26]:
group_total = all_data.groupby("articleID")["Y_preds_label"].count()
group_positive = all_data.groupby("articleID")["Y_preds_label"].sum()
#合併上面兩個項目(上面是serise)
group = pd.DataFrame({'total':group_total, 'positive_total':group_positive})

In [27]:
#把articleID從index取出來
group['articleID'] = group.index
group = group.reset_index(drop = True)

In [28]:
#負面的有幾個
group["negative_total"] = group["total"] - group["positive_total"]

### 合併原始資料和剛剛的計算資料

In [30]:
count_all_data = pd.merge(group,
                          all_data[['articleID', 'url', 'title', 'createTime']],
                          on='articleID',how = 'left')

In [31]:
#刪除重複的資料+重製index
count_all_data = count_all_data.drop_duplicates(subset='articleID', keep='first', inplace=False)
count_all_data = count_all_data.reset_index(drop = True)

### 計算趴數

In [33]:
#趴數
count_all_data["positive_percent"] = count_all_data["positive_total"] / count_all_data["total"]
count_all_data["negative_percent"] = count_all_data["negative_total"] / count_all_data["total"]

### 把他排的整齊一點

In [34]:
count_all_data = count_all_data[['articleID', 'url', 'title', 'createTime', 'positive_total', 'negative_total', 'total', 'positive_percent', 'negative_percent']]

In [36]:
count_all_data.to_csv('./savenocut/count.csv', index=False, encoding='utf-8-sig')

In [45]:
overfive = count_all_data[count_all_data["total"] > 5]

In [42]:
len(count_all_data)

1570

### 取出前3大的欄位

In [46]:
#取出前3大的欄位
positive_top = overfive.nlargest(3,'positive_percent')
negative_top = overfive.nlargest(3,'negative_percent')

### 正向

In [47]:
positive_top["url"].tolist()
positive_top["positive_percent"].tolist()

[1.0, 0.875, 0.8421052631578947]

### 負向

In [48]:
negative_top["url"].tolist()
negative_top["negative_percent"].tolist()

[1.0, 1.0, 1.0]

# 篩選

In [90]:
#篩選他
def create_select_data(data):
    select = input()
    select_id = []
    for i in range(0, len(data)):
        if select in data["title"][i] :
            select_id.append(i)
            #print(i, data["title"][i])
    select_data = data.loc[select_id]
    select_data = select_data.reset_index(drop = True)
    
    return select_data

In [91]:
#幫data加上一個預測完的欄位
def preds_data(data):
    #變成 list
    all_text = data["recovery"].tolist()
    
    #預測他
    X_test = text_to_index(all_text)
    X_test = pad_sequences(X_test, maxlen=PADDING_LENGTH)
    Y_preds = lstm.predict(X_test)
    Y_preds_label = np.argmax(Y_preds, axis=1)
    
    data["Y_preds_label"] = Y_preds_label
    
    return data

In [92]:
#產生 group他計算正負項
#這裡的data必須要有Y_preds_label欄位

def create_count_positivedata_negativedata(data):
    group_total = data.groupby("articleID")["Y_preds_label"].count()
    group_positive = data.groupby("articleID")["Y_preds_label"].sum()
    #合併上面兩個項目(上面是serise)
    group = pd.DataFrame({'total':group_total, 'positive_total':group_positive})
    
    group['articleID'] = group.index
    group = group.reset_index(drop = True)
    
    #負面的有幾個
    group["negative_total"] = group["total"] - group["positive_total"]
    
    count_all_data = pd.merge(group,
                          data[['articleID', 'url', 'title', 'createTime']],
                          on='articleID',how = 'left')
    
    #刪除重複的資料+重製index
    count_all_data = count_all_data.drop_duplicates(subset='articleID', keep='first', inplace=False)
    count_all_data = count_all_data.reset_index(drop = True)
    
    #出現計算趴數欄位
    count_all_data["positive_percent"] = count_all_data["positive_total"] / count_all_data["total"]
    count_all_data["negative_percent"] = count_all_data["negative_total"] / count_all_data["total"]
    
    #排整齊
    count_all_data = count_all_data[['articleID', 'url', 'title', 'createTime', 'positive_total', 'negative_total', 'total', 'positive_percent', 'negative_percent']]
    overfive = count_all_data[count_all_data["total"] > 3]
    
    return overfive

In [53]:
#產出正向前三名的兩個list

def top3_positivedata(data):
    positive_top = data.nlargest(3,'positive_percent')
    
    positive_url = positive_top["url"].tolist()
    positive_percent = positive_top["positive_percent"].tolist()
    
    return positive_url, positive_percent


In [98]:
#產出負向前三名的兩個list

def top3_negativedata(data):
    negative_top = data.nlargest(3,'negative_percent')
    
    negative_url = negative_top["url"].tolist()
    negative_percent = negative_top["negative_percent"].tolist()
    
    return negative_url, negative_percent

In [ ]:
select_data = create_select_data(all_data)
preds = preds_data(select_data)
count = create_count_positivedata_negativedata(preds)

In [68]:
top3_positive_url = top3_positivedata(count)[0]
top3_positive_percent = top3_positivedata(count)[1]

In [71]:
top3_negative_url = top3_negativedata(count)[0]
top3_negative_percent = top3_negativedata(count)[1]

# 合併

In [99]:
#一步到底的感覺
def all_step(data):
    select_data = create_select_data(data)
    preds = preds_data(select_data)
    count = create_count_positivedata_negativedata(preds)
    top3_positive_url = top3_positivedata(count)[0]
    top3_positive_percent = top3_positivedata(count)[1]
    top3_negative_url = top3_negativedata(count)[0]
    top3_negative_percent = top3_negativedata(count)[1]
    
    return top3_positive_url, top3_positive_percent, top3_negative_url, top3_negative_percent

In [100]:
output = all_step(all_data)

信義


<ipython-input-6-0467c45ce11a>:32: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(new_corpus)


In [101]:
print("positive url list：" ,output[0])
print("positive percent list：" ,output[1])

print("========================")

print("negative url list：" ,output[2])
print("negative percent list：" ,output[3])

positive url list： ['https://www.myhousing.com.tw//index.php?option=com_kunena&view=topic&catid=8&id=100493&Itemid=0', 'https://www.mobile01.com/topicdetail.php?f=454&t=6019348', 'https://www.mobile01.com/topicdetail.php?f=454&t=6030435']
positive percent list： [1.0, 0.6666666666666666, 0.5229357798165137]
negative url list： ['https://www.mobile01.com/topicdetail.php?f=454&t=6056969', 'https://www.mobile01.com/topicdetail.php?f=454&t=5943880', 'https://www.mobile01.com/topicdetail.php?f=454&t=5911768']
negative percent list： [0.8, 0.7727272727272727, 0.75]
